In [1]:
import open3d as o3d 
import numpy as np 
from scipy.spatial.transform import Rotation as R

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataset = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(dataset.path)
assert (pcd.has_normals())

## Detect planar patches 

In [3]:
# using all defaults
# This function then returns a list of detected planar patches, represented as geometry::OrientedBoundingBox objects, 
# with the third column (i.e., z) of R indicating the planar patch normal vector.
oboxes = pcd.detect_planar_patches(
    normal_variance_threshold_deg=60,
    coplanarity_deg=75,
    outlier_ratio=0.75,
    min_plane_edge_length=0,
    min_num_points=0,
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

print("Detected {} patches".format(len(oboxes)))

normal_vectors = []
for obox in oboxes:
    normal_vectors.append(obox.R[:, 2])
    

Detected 10 patches


In [6]:
geometries = []
# visualize the patches, bounding boxes, and normals
for i, obox in enumerate(oboxes):
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.1])
    mesh.paint_uniform_color(obox.color)
    normal_mesh = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.02,
        cone_radius=0.05,
        cylinder_height=0.3,
        cone_height=0.1
    )
    # move mesh to obox center
    normal_mesh.translate(obox.get_center())
    # rotate mesh to align with obox normal at the obox center
    o3d.geometry.get_rotation_matrix_from_axis_angle(normal_vectors[i])
    rotaion_matrix = R.align_vectors([normal_vectors[i]], [[0, 0, 1]],)[0].as_matrix()
    normal_mesh.rotate(rotaion_matrix, center=obox.get_center())
    # set normal arrow color to obox color
    normal_mesh.paint_uniform_color(obox.color)
    geometries.append(mesh)
    geometries.append(obox)
    geometries.append(normal_mesh)
geometries.append(pcd)

# also add coords frame 
coords = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
geometries.append(coords)

o3d.visualization.draw_geometries(geometries,
                                  zoom=0.62,
                                  front=[0.4361, -0.2632, -0.8605],
                                  lookat=[2.4947, 1.7728, 1.5541],
                                  up=[-0.1726, -0.9630, 0.2071])

/tmp/ipykernel_77330/3704614.py:16: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotaion_matrix = R.align_vectors([normal_vectors[i]], [[0, 0, 1]],)[0].as_matrix()


## Plane Segmentation using RANSAC 

In [5]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)



Plane equation: -0.05x + -0.10y + 0.99z + -1.08 = 0


In [6]:
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])